 # Sugar tutorial





The objective of Sugar library is to provide a set of handy tools to facilitate the setup of simulations of polyelectrolytes and peptides with the Molecular Dynamics software ESPResSo. In other words: adding sugar to your ESPResSo scripts makes your life sweater. In this tutorial, the different functionalities of sugar are explained with different practical examples.

## Table of contents:
* [Introduction](#introduction)
* [Use Sugar to create particles in ESPResSo](#particles)
* [Use Sugar to create polymers in ESPResSo](#polymers)
* [Use Sugar to create peptides in ESPResSo](#peptides)

## Introduction <a class="anchor" id="introduction"></a>

Let us get started by importing sugar library and creating an instance of it. 

In [1]:
import sugar
sg=sugar.sugar_library()

When sugar is inicialitated, a default system of reduced units is defined. The active set of reduced units can be consulted using

In [2]:
sg.print_reduced_units()


Current set of reduced units:
0.355 nanometer = 1 reduced_length
4.11640356238e-21 joule = 1 reduced_energy
Temperature: 298.15 kelvin
1.602176634e-19 coulomb = 1 reduced_charge



This default definition of reduced units can be changed at the convience of the user using the command

In [3]:
sg.set_reduced_units(unit_length=0.5*sg.units.nm,  unit_charge=2*sg.units.e, temperature=300 * sg.units.K)


Current set of reduced units:
0.49999999999999994 nanometer = 1 reduced_length
4.141945559999999e-21 joule = 1 reduced_energy
Temperature: 300 kelvin
3.204353268e-19 coulomb = 1 reduced_charge



All input variables will be given to ESPResSo using these reduced units, since it is a convinient choice for the simulation setup. Internally, Sugar uses Pint library to deal with unit transformations, which in turn should be  used by the user to define its own variables

In [4]:
Box_L = 7 * sg.units.nm

print('The side of the simulation box is ', Box_L, '=' ,Box_L.to('reduced_length'))

import espressomd

# Create an instance of an ESPResSo system 

system=espressomd.System(box_l=[Box_L.to('reduced_length').magnitude]*3)

The side of the simulation box is  7 nanometer = 13.999999999999998 reduced_length


## Use Sugar to create particles in ESPResSo <a class="anchor" id="particles"></a>

Particles are the smaller objects in the simulation box, which can represent small ions or other small chemical species. In turn, particles can also be used as building blocks for larger molecules or polymers, where they can represent one monomeric unit or part of it. Particle objects are used in sugar as input for several of its funcionalities, including to create larger molecules and peptides. The basic properties of a particle are:

In [5]:

cation=sg.particle(name='Na', 
                   type=0, 
                   q=1, 
                   diameter=0.35*sg.units.nm, 
                   epsilon=1*sg.units('reduced_energy'))

print('label for the particle', cation.name)
print('type of the particle to be identified within espresso', cation.type)
print('charge of the particle', cation.q)

# These parameters are only requiered for posterior setting up of Lennard-Jones interactions 

print('diameter of the particle', cation.diameter)
print('epsilon of the particle', cation.epsilon)


label for the particle Na
type of the particle to be identified within espresso 0
charge of the particle 1
diameter of the particle 0.35 nanometer
epsilon of the particle 1 reduced_energy


One can use sugar to create any number of the defined particles into the espresso system

In [6]:
N_cations=20

for _ in range(N_cations):
    sg.create_object_in_system(object=cation, system=system)

Let us see the particles that we have created by visualizing our espresso system

In [7]:
picture_name='cation_system.png'
sg.do_snapshot_system(system=system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
sg.destroy_object_in_system(object=cation, system=system)

## Use Sugar to create polymers in ESPResSo <a class="anchor" id="polymers"></a>

Sugar can be used to easily construct coarse-grained models of polymers with complex structures. Let us consider a coarse grained model for polyhydroalanaline which represents a monomer with three beads, as depicted in the schematics below: a backbone bead (purple), a bead for the carboxylic acid group (cyan) and a bead for the amino group (blue).|
<img src="PDha.png" width=100 height=100 />


To set up such polymer with sugar first one has to define the different particles in the monomer

In [8]:
# Define each different bead in the polymer
# Use sg.propose_unused_type() to let sugar choose one non-used particle type for you

backbone_bead=sg.particle(name='C', 
                   type=sg.propose_unused_type(),
                   q=0, 
                   diameter=0.4*sg.units.nm, 
                   epsilon=1*sg.units('reduced_energy'))

carboxyl_bead=sg.particle(name='COOH', 
                   type=sg.propose_unused_type(),
                   q=0, 
                   diameter=0.5*sg.units.nm, 
                   epsilon=1*sg.units('reduced_energy'))

amine_bead=sg.particle(name='NH3', 
                   type=sg.propose_unused_type(),
                   q=0, 
                   diameter=0.3*sg.units.nm, 
                   epsilon=1*sg.units('reduced_energy'))


Then, one defines the structure of the residue of the polymer. A residue is composed by a `central_bead` where one or various `side_chains` are attached. Each side chain can contain one particle or other residues. 

In [9]:
# Define the monomer residue

PDha_residue=sg.residue(name='PDha_mon',
                    central_bead=backbone_bead,
                    side_chains=[carboxyl_bead,amine_bead])


Once done, one has to define a bond for each different bond in the polymer. For simplicity, in this tutorial we assume that all bonds are equal and we set-up all bonds using the same generic bond

In [10]:
# Define the bonds between the particles

from espressomd import interactions

generic_bond_lenght=0.4 * sg.units.nm
generic_harmonic_constant=400*sg.units('reduced_energy / nm**2') 
generic_bond = interactions.HarmonicBond(k=generic_harmonic_constant.to('reduced_energy / reduced_length**2').magnitude,
                                 r_0=generic_bond_lenght.to('reduced_length').magnitude)

# C-C bond
sg.define_bond(bond=generic_bond, particle1=backbone_bead, particle2=backbone_bead)
# C-COOH bond
sg.define_bond(bond=generic_bond, particle1=backbone_bead, particle2=carboxyl_bead)
# C-NH4 bond
sg.define_bond(bond=generic_bond, particle1=backbone_bead, particle2=amine_bead)

sg.add_bonds_to_system(system=system)

Finally, one uses the residues to define the polymer sequence given by the argument `residue_list`. Note that one needs to add one residue in `residue_list` per each residue in your polymer chain, i.e.

In [11]:
# Define the polymer molecule

N_monomers=8
PDha_polymer=sg.molecule(name='PDha',residue_list=[PDha_residue]*N_monomers)

# Create a PdHa dimer into the espresso system

N_pol=1

for _ in range(N_pol):
    sg.create_object_in_system(object=PDha_polymer, 
                               system=system, 
                               position=[Box_L.to('reduced_length').magnitude/2]*3) #Create it in the box center 

picture_name='PDha_system.png'
sg.do_snapshot_system(system=system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
sg.destroy_object_in_system(object=PDha_polymer, system=system)

## Use Sugar to create peptides <a class="anchor" id="peptides"></a>

Sugar includes built-on functions to facilitate the setting up of coarse-grained models for peptides from their aminoacid sequence. At the moment, two different coarse-grained models have been implemented with one bead (`model=1beadAA`) or two beads (`model=2beadAA`) per aminoacid. For the aminoacids currently available, we provide reference parameters in the folder (`reference_parameters`) which can be loaded into Sugar

In [12]:
N_peptide=1
sequence="KKKKKEEEEE"
model='2beadAA'  # Model with 2 beads per each aminoacid

 # Load peptide parametrization from Lunkad, R. et al.  Molecular Systems Design & Engineering (2021), 6(2), 122-131.

sg.load_parameters(filename='reference_parameters/Lunkad2021.txt')
sg.add_bonds_to_system(system=system) # Add the bonds loaded from the file to the espresso system

peptide = sg.peptide(name=sequence, sequence=sequence, model=model)

for _ in range(N_peptide):
    sg.create_object_in_system(object=peptide, 
                               system=system, 
                               position=[Box_L.to('reduced_length').magnitude/2]*3) #Create it in the box center 
    
picture_name='peptide_system.png'
sg.do_snapshot_system(system=system, filename=picture_name)

# To show the picture
from PIL import Image
img = Image.open(picture_name)
img.show()

# To clean-up the system before continuing the tutorial
sg.destroy_object_in_system(object=peptide, system=system)

Note that in `sequence` we have used the one letter code for the aminoacids. Internally, Sugar uses the one-letter code to define each aminoacid. The three letter code can also be used as input, which Sugar automatically detects and transforms into the one-letter code using its own protein sequence parser.

In [13]:
# The same peptide as before, but now using the three letter code for aminoacids

sequence='LYS-LYS-LYS-LYS-LYS-GLU-GLU-GLU-GLU-GLU'
peptide = sg.peptide(name=sequence, sequence=sequence, model=model)
print('one letter code', sg.protein_sequence_parser(sequence=sequence))
print('defined peptide sequence ', peptide.sequence)


one letter code ['K', 'K', 'K', 'K', 'K', 'E', 'E', 'E', 'E', 'E']
defined peptide sequence  ['K', 'K', 'K', 'K', 'K', 'E', 'E', 'E', 'E', 'E']
